# Python defaultdict 详解  
## 1. defaultdict 简介  
`defaultdict` 是 Python `collections` 模块提供的一种特殊字典类型，它是内置 `dict` 类的子类。主要特点是**当访问不存在的键时，不会引发 `KeyError` 异常**，而是自动为该键创建一个默认值 [7](@ref)。  
**核心机制**：通过 `default_factory` 参数指定一个可调用对象（工厂函数），当键不存在时，自动调用此函数生成默认值 [1,6](@ref)。  

## 2. 常用默认工厂函数

| 工厂函数 | 生成的默认值 | 典型应用场景 |
|---------|------------|------------|
| `list` | 空列表 `[]` | 数据分组，将一系列元素归到不同键下 [2,6](@ref) |
| `int` | 整数 `0` | 计数或频率统计 [3,6](@ref) |
| `set` | 空集合 `set()` | 需要元素去重的分组场景 [4](@ref) |
| `dict` | 空字典 `{}` | 创建多层嵌套字典 [3,4](@ref) |
| `str` | 空字符串 `''` | 字符串拼接初始化 [3](@ref) |
| `lambda` 表达式 | 自定义的任意值 | 需要复杂初始数据结构时 [1,2](@ref) |

## 3. 具体用法示例

### 3.1 使用 list 作为默认工厂
**场景**：按组归类数据

In [2]:
from collections import defaultdict

names = [('group1', 'Alice'), ('group2', 'Bob'), ('group1', 'Charlie')]
grouped_names = defaultdict(list)
for group, name in names:
    grouped_names[group].append(name) # 无需判断键是否存在
print(grouped_names)

defaultdict(<class 'list'>, {'group1': ['Alice', 'Charlie'], 'group2': ['Bob']})


### 3.2 使用 int 作为默认工厂  
**场景**：计数统计 统计元素出现次数

In [3]:
bag = ["apple", "orange", "cherry", "apple", "cherry", "blueberry"]
count = defaultdict(int)
for item in bag:
    count[item] += 1 # 不存在的键自动初始化为0
print(count)

defaultdict(<class 'int'>, {'apple': 2, 'orange': 1, 'cherry': 2, 'blueberry': 1})


### 3.3 使用 set 作为默认工厂
**场景**：需要去重的分组

In [4]:
dict_set = defaultdict(set)
dict_set['a'].add(1)
dict_set['a'].update([2, 3]) # 自动去重
print(dict_set)

defaultdict(<class 'set'>, {'a': {1, 2, 3}})


### 3.4 使用自定义函数/lambda
**场景**：需要特定的默认值 使用lambda设置特定默认值

In [5]:
scores = defaultdict(lambda: 1500) # 默认分数1500
scores['xiaoming'] += 10 # 从1510开始
print(scores['xiaoming']) # 输出: 1510

1510


使用自定义函数返回复杂默认值

In [6]:
def complex_default():
    return {'count': 0, 'total': 0.0}
data = defaultdict(complex_default)
data['key1']['count'] += 1

### 3.5 创建嵌套字典
**场景**：多层数据结构 自动创建多层嵌套字典

In [ ]:
nested_dict = defaultdict(lambda: defaultdict(int))
nested_dict['outer_key']['inner_key'] += 1 # 自动创建两层结构

或者使用常规defaultdict嵌套

In [ ]:
dict_of_dicts = defaultdict(dict)
dict_of_dicts['k1'].update({"name": "John"})

### 4.3 重要注意事项
1. **工厂函数必须是可调用对象**：传入的是类/函数，不是具体值
2. **避免可变默认值的引用陷阱**：

<font color=#FF3366>错误：所有键共享同一个列表</font>

In [ ]:
dd = defaultdict(lambda: [100])

<font color=#00FF00>正确：每次生成新列表</font>  
虽然可以直接设置`defaultdict(lambda: [100])`，但这会导致所有新键的初始值都是同一个列表对象的引用。修改其中一个键的列表（如`dd['a'].append(200)`），所有共享该默认值的键都会受影响。安全的做法是让工厂函数每次返回一个新对象，这正是`list`、`int`等所做的

In [ ]:
dd = defaultdict(list)

3. **默认值创建时机**：只在通过 `dd[key]` 访问时创建，使用 `get()` 方法不会触发 [8](@ref)
4. **转换为普通字典**：`dict(my_defaultdict)`

## 5. 内部原理简介

`defaultdict` 通过重写 `__missing__` 方法实现自动默认值 [8](@ref)：

```python
python
def missing(self, key):
    if self.default_factory is None:
        raise KeyError(key)
    self[key] = value = self.default_factory()
    return value
```

这意味着默认值只在通过 `d[key]` 方式访问时创建，使用 `d.get(key)` 不会触发此机制。

## 6. 总结

`defaultdict` 通过自动处理键的初始化，让代码更关注业务逻辑而非底层检查，特别适用于分组、统计和嵌套数据结构场景。正确使用可以显著提升代码的简洁性和可读性。